In [29]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

In [30]:
nucleiPath = "segmentations/cellSize/tiles/tilesNuclei"
areaPath = "results/density/densityAllAreas20230214.csv"
saveName = "results/density/densityPartsNew_20230215.csv"

In [32]:
folders = [x[0] for x in os.walk(nucleiPath)]
folders = sorted(folders)

In [ ]:
#calculate density for every image tile
df = pd.read_csv(areaPath)
dfResults = pd.DataFrame()
for index, row in df.iterrows():
    image = row['Label']
    area = row['Area']
    age = image.split("_")[1]
    aorta = image.split("_")[2]
    nucFile = image.split(".tif")[0] + "_seg.npy"
    nucPath = join(nucleiPath, age, aorta, nucFile)
    nucleiMasks =  np.load(nucPath, allow_pickle=True).item()['masks']
    maskNumber = len(np.unique(nucleiMasks)) - 1
    density = maskNumber/area
    dfHelp = pd.DataFrame({"age": age, "aorta": aorta, "numberNuclei": maskNumber, "area": area, "density": density, "file": image}, index=[0])
    dfResults = dfResults.append(dfHelp)

#convert pixel^2 to micrometer^2
dfResults["densityMicro"] = dfResults["density"]*1.162803946*1.162803946
dfResults.to_csv(saveName)